In [39]:
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import *
from pymatgen.io.cif import *
from pymatgen.core.operations import *
from tabulate import tabulate
from pymatgen.symmetry.site_symmetries import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pymatgen.analysis.diffraction.xrd import *
from pymatgen.electronic_structure import bandstructure
from pymatgen.electronic_structure import plotter

In [40]:
USER_API_KEY = "VE2Z5ZyFtdd4NPJ6F"
with MPRester(USER_API_KEY) as m:
    structure = m.get_structure_by_material_id("mp-568662")
    bandstructure = m.get_bandstructure_by_material_id("mp-568662")

In [41]:
bandgap = bandstructure.get_band_gap()

## Déterminer la bande interdite

In [42]:
print("Énergie de la bande interdite : ",str(bandgap.get('energy'))+" eV")
print("Bande interdite directe ? : ", str(bandgap.get('direct')))

Énergie de la bande interdite :  5.4291 eV
Bande interdite directe ? :  False


La bande interdite n'est pas directe et est supérieur à 1-2 eV, on est dans le cas d'un matériaux isolant 
(from https://fr.wikipedia.org/wiki/Théorie_des_bandes)

## Trouver la direction dans laquelle la dernière (première) bande de valence (conduction) présente le plus et le moins de dispersion (insérer une figure indiquant la dispersion par des flèches),

Rappels : 
- Conduction Band Minimum (CBM) = bande contenant le minimum de la bande de valence
- Valence Band Maximum (VBM) =  bande contenant le maximum de la bande de valence


In [47]:
cbm = bandstructure.get_cbm()
print("Premiere bande de conduction ")
print("Bande contenant le CBM",str(cbm.get('band_index')))

Premiere bande de conduction 
Bande contenant le CBM defaultdict(<class 'list'>, {<Spin.up: 1>: [12]})


#### La bande 12 est donc la première bande de valence.

In [44]:
vbm = bandstructure.get_vbm()
print("Derniere bande de condution")
print("Bande contenant le VBM : ",str(vbm.get('band_index')))

Derniere bande de condution
Bande contenant le VBM :  defaultdict(<class 'list'>, {<Spin.up: 1>: [11]})


#### La bande 11 est donc la dernière bande de valence.

#################################################################################################################

## Calculer la masse effective au sommet (base) de la dernière (première) bande de valence (conduction) en supposant une dispersion parabolique

In [58]:
data = plot.bs_plot_data()
distance = data.get('ticks').get('distance')
distances = data.get('distances')
energy = data.get('energy')